In [26]:
#pip install --upgrade pandas
#pip install pyspark==4.5.1

In [ ]:
# bibliotecas necessárias
import os
import sys
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import date_format, col

# Parâmetros de conexão com PostgreSQL
host = "dpg-d2q93uv5r7bs73aesqlg-a.oregon-postgres.render.com"
port = 5432
database = "sicoop"
user = "admin"
password = "vnbygROdVav9q0C5gqF6S7zJjieM7hIr"

def create_spark_session():
    
    print("\nCriando SparkSession...")

    spark = SparkSession.builder \
        .appName("ETL_Sicoop_PostgreSQL") \
        .master("local[*]") \
        .config("spark.driver.extraClassPath", "drivers/postgresql-42.7.7.jar") \
        .config("spark.executor.extraClassPath", "drivers/postgresql-42.7.7.jar") \
        .config("spark.driver.extraJavaOptions", "-Djava.io.tmpdir=C:/temp") \
        .config("spark.executor.extraJavaOptions", "-Djava.io.tmpdir=C:/temp") \
        .config("spark.sql.adaptive.enabled", "false") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "false") \
        .getOrCreate()
        
    return spark

def configure_postgres_connection():
    
    print("\nConexão PostgreSQL...")
    
    jdbc_url = f"jdbc:postgresql://{host}:{port}/{database}"
    connection_properties = {
        "user": user,
        "password": password,
        "driver": "org.postgresql.Driver"
    }
        
    return jdbc_url, connection_properties

def read_table_from_postgres(spark, jdbc_url, connection_properties, table_name):
    
    print(f"Inicia Leitura do Script...")

    df = spark.read.jdbc(
        url=jdbc_url, 
        table=table_name, 
        properties=connection_properties
    )
    
    print(f"Número de linhas retornadas: {df.count()}")
    
    return df


In [ ]:
# Cria SparkSession
spark = create_spark_session()

# Configurar conexão PostgreSQL
jdbc_url, connection_properties = configure_postgres_connection()

query = """
(
    select 	
        ass.nome              nome_associado,
        ass.sobrenome         sobrenome_associado,
        ass.idade             idade_associado,
        sum(mov.vlr_transacao) vlr_transacao_movimento,
        mov.des_tranacao      des_tranacao_movimento,
        mov.data_movimento    data_movimento,
        crt.num_cartao        numero_cartao,
        crt.nom_impresso      nome_impresso_cartao,
        crt.data_criacao      data_criacao_cartao,
        cnt.tipo              tipo_conta,
        cnt.data_criacao      data_criacao_conta
    from 	
        movimento mov
        inner join cartao crt
            on mov.id_cartao = crt.id_cartao 
        inner join conta cnt
            on crt.id_conta  = cnt.id_conta
        inner join associado ass
            on cnt.id_associado = ass.id_associado 
    group by
        ass.nome,
        ass.sobrenome,
        ass.idade,
        mov.des_tranacao,
        mov.data_movimento,
        crt.num_cartao,
        crt.nom_impresso,
        crt.data_criacao,
        cnt.tipo,
        cnt.data_criacao
) as consulta
"""

df_movimento =  read_table_from_postgres(spark, jdbc_url, connection_properties, query)


Criando SparkSession...

Conexão PostgreSQL...
Inicia Leitura do Script...
Número de linhas retornadas: 1000


In [30]:
# Converter timestamps para string no Spark
df_movimento_treated = df_movimento
for col_name in df_movimento.columns:
    if 'data' in col_name.lower():
        df_movimento_treated = df_movimento_treated.withColumn(col_name, date_format(col(col_name), 'yyyy-MM-dd HH:mm:ss'))
        
# Converter para Pandas
df_movimento_output = df_movimento_treated.toPandas()
        
# Salvar CSV
output_path = "output/movimento_final.csv"
df_movimento_output.to_csv(output_path, sep=';', index=False, header=True, encoding='utf-8')